In [74]:
import string
import re
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import math
import glob

In [83]:
with open('raw.txt', 'r') as myfile:
    data=myfile.read().replace('\n', ' ')

In [113]:
path_fed = '/Users/arvenkat/Documents/TextAnalyticsMSIA/HW4/fed_papers/*.txt'



keys = [1,6,8,9,12]
texts = []
count = 0
fed_dct.fromkeys(keys)
for filename in glob.glob(path_fed):
    with open(filename) as infile:
        datum=infile.read().replace('\n', ' ')
        texts.append(datum)
fed_dct = dict(zip(keys, texts))
    

In [85]:
charlist = ["BURR","HAMILTON","MADISON","LAURENS", "MULLIGAN", "ENSEMBLE", "WASHINGTON","COMPANY","ANGELICA","ELIZA",
           "LAFAYETTE","JEFFERSON"]

In [86]:
exclude = set(string.punctuation)
data = ''.join(ch for ch in data if ch not in exclude)

excludeslash = ("/")
data = ''.join(ch for ch in data if ch not in excludeslash)

In [87]:
def get_char_files(chrlist,dat):
    chardct = {}
    list_of_txts = []
    iterator = 0
    for char in chrlist:
        chrflag = False
        chrstring = ""
        for word in dat.split():
            if char in word:
                chrflag = True
            if word.isupper() and word != char:
                chrflag = False
            if chrflag and word != char:
                chrstring = chrstring + word + " "
        list_of_txts.append(chrstring)
    for value in chrlist:
        chardct[value] = list_of_txts[iterator]
        iterator = iterator + 1
    return chardct


In [88]:
dct = get_char_files(charlist,data)

In [89]:
dct

{'ANGELICA': 'Angelica Angelica Daddy doesn’t need to know But—look around look around the Revolution’s happening in New York New ideas in the air Eliza I’m lookin’ for a mind at work I’m lookin’ for a mind at work I’m lookin’ for a mind at work Whooaaaaa Burr you disgust me I’ve been reading Common Sense by Thomas Paine So men say that I’m intense or I’m insane You want a revolution And when I’m ‘a compel him to include women in the sequel Cuz I’ve been reading Common Sense by Thomas Paine So men say that I’m intense or I’m insane You want a revolution Angelica Whoa In the greatest City in the world I’m about to change your life My sister I’ll leave you to it I’m just sayin’ if you really loved me you would share him I’m sure Is that right My name is Angelica Schuyler Where’s your fam’ly from So so so— So this is what it feels like to match wits With someone at your level What the hell is the catch It’s The feeling of freedom of seein’ the light It’s Ben Franklin with a key and a kite

In [90]:
#Count vocabulary Size and number of tokens
tokens = 0
unique_word_list = []
for lyric in dct.values():
    tokens += len(lyric.split())
    for word in lyric.split():
        if word not in unique_word_list:
            unique_word_list.append(word)
            
    
print("Tokens: (%i)" % tokens)
print("Vocabulary Size: (%i)" % len(unique_word_list))

Tokens: (10534)
Vocabulary Size: (2370)


In [91]:
###TESTING####
test = "A voice saying WASHINGTON “You gotta fend for yourself”\tCOMPANY “Alex you gotta fend for yourself” WASHINGTON He started retreatin’ and readin’ every treatise on the shelf BURR There would have been nothin’ left to do For someone less astute He woulda been dead or destitute Without a cent of restitution Started workin’ clerkin’ for his late mother’s landlord Tradin’ sugar cane and rum and all the things he can’t afford Scammin’ for every book he can get his hands on Plannin’ for the future see him now as he stands on The bow of a ship headed for a new land In New York you can be a new man COMPANY Scammin’ Plannin’ Oooh… COMPANY In New York you can Be a new man— In New York you can Be a new man— HAMILTON Just you wait Just you wait COMPANY In New York you can be a new man— WOMEN In New York— MEN New York— HAMILTON Just you wait COMPANY Alexander Hamilton We are waiting in the wings for you You could never back down You never learned to take your time Oh Alexander Hamilton When America sings for you Will they know what you overcame Will they know you rewrote the game The world will never be the same oh BURR The ship is in the harbor now See if you can spot him Another immigrant Comin’ up from the bottom His enemies destroyed his rep America forgot him\tCOMPANY Alexander Hamilton Waiting in the wings for you You never learned to take your time Oh Alexander Hamilton Alexander Hamilton… America sings for you Will they know what you overcame Will they know you rewrote the game The world will never be the same oh MEN Just you wait COMPANY"
chrflag = False
chrstring = ""
for word in test.split():
    if word == "WASHINGTON":
        chrflag = True
    if word.isupper() and word != "WASHINGTON":
        chrflag = False
    if chrflag and word != "WASHINGTON":
        chrstring = chrstring + word + " "
   



In [92]:
from pycorenlp import StanfordCoreNLP
nlp = StanfordCoreNLP('http://localhost:9000')

In [93]:
#loop over this
def normalize_bios(dictionary):
    for val in dictionary:
        lemmatize = nlp.annotate(dictionary[val], properties={
                'annotators': 'tokenize,lemma',
                'outputFormat': 'json',
                'timeout': 100000,
            })




        sent = ''
        for s in lemmatize["sentences"]:
            sentence = str(" ".join([t["lemma"] for t in s["tokens"]]))
            sent = sent + sentence
        dictionary[val] = sent
    

In [94]:
normalize_bios(dct)
normalize_bios(fed_dct)

In [95]:
stopWords = set(stopwords.words('english'))
exclude = set(string.punctuation)

def preprocess(dictionary):
    for key in dictionary:
        text = ' '.join([word for word in dictionary[key].split() if word not in stopwords.words("english")])
        s = ''.join(ch for ch in text if ch not in exclude)
        r = ''.join([i for i in s if not i.isdigit()])
        dictionary[key] = r
    

In [96]:
preprocess(dct)
preprocess(fed_dct)

In [97]:
def get_words(dictionary):
    words = []
    for val in dictionary:
        spt = dictionary[val].split()
        set_couples = set(spt)
        for word in set_couples:
            words.append(word)
    words = list(set(words))
    return words

In [98]:
#similarity stuff




def calc_frequency(row,col,dictionary):
    doc_term = np.zeros(shape=(row,col))
    for i in range(0,row):
        j = 0
        for key in dictionary:
            word = get_words(dictionary)[i]
            bio = dictionary[key]
            doc_term[i][j] = bio.count(word)
            j = j + 1
    return doc_term





def tf_idf_transform(d,row,col):
    tfidf_term = np.zeros(shape=(row,col))
    big_n = 12
    for i in range(0,row):
        for j in range(0,col):
            val = d.get_value(i,j)
            if val != 0:
                tf = val
                #print (tf)
                y = np.array(d.loc[i])
                idf = np.count_nonzero(y > 0)
                tfidf = (1+math.log(tf))*(math.log(big_n/idf))   
                tfidf_term[i][j] = tfidf
    return tfidf_term



def cosine_similarity(v1,v2):
    "compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/math.sqrt(sumxx*sumyy)

def calc_sim(doc,p,n):
    sim_term = np.zeros(shape=(n,n))
    for i in range(0,n):
        for j in range(0,n):
            if i == j:
                sim_term[i][j] = 0
            else:
                sim_term[i][j] = cosine_similarity(doc.iloc[:,i],doc.iloc[:,j])
    return sim_term

In [100]:


df_freq_h = pd.DataFrame(calc_frequency(1642,12,dct))
df_tfidf_h = tf_idf_transform(df_freq_h,1642,12)
df_tfidf_h = pd.DataFrame(df_tfidf_h)
tf_idfmat_h = pd.DataFrame(calc_sim(df_tfidf_h,1642,12))

In [120]:

tf_idfmat_h.columns = charlist
tf_idfmat_h.set_index(tf_idfmat_h.columns)

,BURR,HAMILTON,MADISON,LAURENS,MULLIGAN,ENSEMBLE,WASHINGTON,COMPANY,ANGELICA,ELIZA,LAFAYETTE,JEFFERSON
BURR,0.000000,0.235595,0.119011,0.088778,0.039648,0.116591,0.124569,0.228233,0.115091,0.094264,0.080702,0.146439
HAMILTON,0.235595,0.000000,0.117268,0.121909,0.036697,0.088122,0.193894,0.138331,0.133696,0.115554,0.078209,0.137801
MADISON,0.119011,0.117268,0.000000,0.030666,0.017988,0.038709,0.051113,0.037071,0.046541,0.030161,0.016303,0.107249
LAURENS,0.088778,0.121909,0.030666,0.000000,0.069868,0.077504,0.063623,0.067464,0.071594,0.058740,0.073926,0.106610
MULLIGAN,0.039648,0.036697,0.017988,0.069868,0.000000,0.028731,0.036839,0.049627,0.009484,0.012195,0.050821,0.029170
ENSEMBLE,0.116591,0.088122,0.038709,0.077504,0.028731,0.000000,0.059774,0.175047,0.048725,0.037077,0.056204,0.062520
WASHINGTON,0.124569,0.193894,0.051113,0.063623,0.036839,0.059774,0.000000,0.125206,0.055123,0.069697,0.071995,0.107195
COMPANY,0.228233,0.138331,0.037071,0.067464,0.049627,0.175047,0.125206,0.000000,0.098789,0.077413,0.046288,0.088203
ANGELICA,0.115091,0.133696,0.046541,0.071594,0.009484,0.048725,0.055123,0.098789,0.000000,0.140324,0.032063,0.072715
ELIZA,0.094264,0.115554,0.030161,0.058740,0.012195,0.037077,0.069697,0.077413,0.140324,0.000000,0.012986,0.085246


In [126]:
tf_idfmat_f.to_csv('ham.csv')
tf_idfmat_h.to_csv('fed.csv')

In [114]:
fed_dct["Hamilton"] = dct["HAMILTON"]

In [117]:
df_freq_f = pd.DataFrame(calc_frequency(2317,6,fed_dct))
df_tfidf_f = tf_idf_transform(df_freq_f,2317,6)
df_tfidf_f = pd.DataFrame(df_tfidf_f)
tf_idfmat_f = pd.DataFrame(calc_sim(df_tfidf_f,2317,6))
tf_idfmat_f.columns = ["1","6","8","9","12","Hamilton"]

In [118]:

tf_idfmat_f

,1,6,8,9,12,Hamilton
0,0.000000,0.293087,0.290139,0.295491,0.311094,0.173158
1,0.293087,0.000000,0.299823,0.328430,0.286338,0.163391
2,0.290139,0.299823,0.000000,0.306694,0.307962,0.163054
3,0.295491,0.328430,0.306694,0.000000,0.317595,0.178236
4,0.311094,0.286338,0.307962,0.317595,0.000000,0.166273
5,0.173158,0.163391,0.163054,0.178236,0.166273,0.000000


In [119]:
tf_idfmat_h

,BURR,HAMILTON,MADISON,LAURENS,MULLIGAN,ENSEMBLE,WASHINGTON,COMPANY,ANGELICA,ELIZA,LAFAYETTE,JEFFERSON
0,0.000000,0.235595,0.119011,0.088778,0.039648,0.116591,0.124569,0.228233,0.115091,0.094264,0.080702,0.146439
1,0.235595,0.000000,0.117268,0.121909,0.036697,0.088122,0.193894,0.138331,0.133696,0.115554,0.078209,0.137801
2,0.119011,0.117268,0.000000,0.030666,0.017988,0.038709,0.051113,0.037071,0.046541,0.030161,0.016303,0.107249
3,0.088778,0.121909,0.030666,0.000000,0.069868,0.077504,0.063623,0.067464,0.071594,0.058740,0.073926,0.106610
4,0.039648,0.036697,0.017988,0.069868,0.000000,0.028731,0.036839,0.049627,0.009484,0.012195,0.050821,0.029170
5,0.116591,0.088122,0.038709,0.077504,0.028731,0.000000,0.059774,0.175047,0.048725,0.037077,0.056204,0.062520
6,0.124569,0.193894,0.051113,0.063623,0.036839,0.059774,0.000000,0.125206,0.055123,0.069697,0.071995,0.107195
7,0.228233,0.138331,0.037071,0.067464,0.049627,0.175047,0.125206,0.000000,0.098789,0.077413,0.046288,0.088203
8,0.115091,0.133696,0.046541,0.071594,0.009484,0.048725,0.055123,0.098789,0.000000,0.140324,0.032063,0.072715
9,0.094264,0.115554,0.030161,0.058740,0.012195,0.037077,0.069697,0.077413,0.140324,0.000000,0.012986,0.085246


In [4]:
import nltk
from nltk.tokenize import PunktSentenceTokenizer
#renormalize - tokenize in NLTK for consistency

#named entities for the first song

custom_s = PunktSentenceTokenizer(data)
tokenized = custom_s.tokenize(data)

tokenized_samp = tokenized[0][0:3450]

NameError: name 'data' is not defined

In [190]:
def entities(text):
    return nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(text)))
        
            
           

In [191]:
with open('no_speakers.txt', 'r') as myfile:
    nochar =myfile.read().replace('\n', ' ')

In [193]:
tree = entities(nochar[0:4000])
tree.draw()

In [128]:
place_list = ["Harlem","America","Princeton","New York City", "Carribean"]

In [200]:
nyc_counts = [0]*len(nochar.split())
ny_counts = [0]*len(nochar.split())

In [201]:
def find_place_nyc(txt,word,checklist):
    splits = txt.split()
    for i in range(len(splits)-1):
        check = splits[i]
        nextw = splits[i+1]
        if check == "York" and nextw == "City":
            print (i, word)
            checklist[i] = 1
            checklist[i-1] = 1
            checklist[i+1] = 1
    return checklist


def find_place_ny(txt,word,checklist):
    splits = txt.split()
    for i in range(1,len(splits)-1):
        check = splits[i]
        nextw = splits[i+1]
        if check == "York" and nextw != "City":
            print (i, word)
            checklist[i] = 1
            checklist[i-1] = 1
    return checklist

In [202]:
nyc_counts = find_place_nyc(nochar,"York",nyc_counts)
ny_counts = find_place_ny(nochar,"York",ny_counts)

603 York
1094 York
5818 York
14052 York
21154 York
377 York
389 York
398 York
413 York
2415 York
2417 York
3514 York
3521 York
3542 York
7115 York
9700 York
9709 York
10010 York
11208 York
11436 York
14538 York
14540 York
16390 York
16972 York


In [203]:
df_ts = np.zeros(shape=(len(nochar.split()),2))

In [204]:
df_ts[:,0] = nyc_counts
df_ts[:,1] = ny_counts
df_ts = pd.DataFrame(df_ts)
df_ts.columns = ["NYC","NY"]

In [205]:
df_ts.to_csv("df_ts.csv")

In [206]:
df_ts

,NYC,NY
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
5,0.0,0.0
6,0.0,0.0
7,0.0,0.0
8,0.0,0.0
9,0.0,0.0
